<p style="text-align:center">
PSY 394U <b>Data Analytics with Python</b>, Spring 2018


<img style="width: 400px; padding: 0px;" src="https://github.com/sathayas/JupyterAnalyticsSpring2018/blob/master/images/Title_pics.png?raw=true" alt="title pics"/>

</p>

<p style="text-align:center; font-size:40px; margin-bottom: 30px;"><b> Logistic regression </b></p>

<p style="text-align:center; font-size:18px; margin-bottom: 32px;"><b>February 13, 2018</b></p>

<hr style="height:5px;border:none" />

# 1. What is a linear discriminant analysis?
<hr style="height:1px;border:none" />

In a typical linear regression, the dependent variable $Y$ is expressed as a linear combination of independent variables $X_1$, $X_2$, ... $X_p$

$$Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \cdots + \beta_p X_p + \epsilon$$

Here, $\beta$s are unknown regression coefficients to be estimated, and $\epsilon$ is the error assumed to follow a normal distribution. This model works well if $Y$ is a continuous variable. But say if $Y$ is a binary variable, taking values either 0 or 1. In that case, the linear model above does not work well since Y is not exactly 0 or 1. 

The solution to modeling a binary outcome variable is to use a logistic regression. Rather than modeling $Y$ directly, it models the probability that $Y$ is 1, or $Pr(Y=1)$. A logistic regression model has the following form:

$$\log\left(\frac{p}{1-p}\right) = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \cdots + \beta_p X_p + \epsilon$$

Or

$$p = \frac{1}{1+e^{-(\beta_0 + \beta_1 X_1 + \beta_2 X_2 + \cdots + \beta_p X_p)}}$$

where $p=Pr(Y=1)$. The function $\log\left(\frac{p}{1-p}\right)$ is referred as a **logit** function. It converts a probability (between 0 and 1) into a real number (between $-\infty$ and $\infty$). With this model, the probability $p=Pr(Y=1)$ can be modeled as a function of independent variables $X$s. Here is an example of the probability $p=Pr(Y=1)$ as it relates to an independent variable $X$ in a logistic regression.

<img style="width: 500px; padding: 0px;" src="https://github.com/sathayas/JupyterAnalyticsSpring2018/blob/master/images/Logistic_LogitFunc.png?raw=true" alt="Logistic function"/>

A logistic regression model can be used ... Pr>0.5 etc.